In [1]:
Assumptions:
    - Every Song belong to an album. A random albumid is generated incase the song is released as single
    - An album can contain songs from multiple artists and so each song can belong to multiple genre. Song is being taken as the lowest degree of transaction 

In [ ]:
# 1. When logged into the reporting portal, one should be able to see all the music a customer has
# purchased with the Company and be able to view the order information (genre, artist, album,
# publication date, order date, price, etc.). Please create a database design to track what a client
# has purchased. The design should be based on a Big Data warehouse environment and embrace
# performance, usability, and minimal storage consumption.

Lake Tables:
    - Customers
        -- customerid - PK
        -- customername
        -- and other customer related data
    - Purchases
        -- transactionid - PK
        -- customerid -FK
        -- songid - FK
        -- spendamount
        -- transactiondate
    - Albums (Dimentional table - One album can have multiple songs, so there will be one to many relationship b/w albumid and songid)
        -- albumid  - PK
        -- albumname 
        -- songid - PK
        -- songname
        -- genere
        -- artist
        -- publicationdate
        -- price 

Mart Table: Based on transactions: This table will be refreshed weekly/daily depending on the number of transactions done. Apply SCD type 2 on customers to update tge other details
    - CustomerPurchases
        -- Customerid -PK
        -- songspurchasedid - PK
        -- spend
        -- purchasedate
        -- genre
        -- artist
    

In [ ]:
# 2. For your design in #1, write a query to find all the music owned by a given client. How do you
# tune and optimize your query?

select customerid
from customerpurchases
where customerid = <customerid>

In [ ]:
#3.  Write an Impala query to find all the clients, who have purchased a song from artist “Barbeque
# Spaceflight” but have never purchased a song from artist “Love Fist”.

select customerid 
from customerpurchases
where artist = "“Barbeque Spaceflight" 
and customerid not in (select customerid from customerpurchases where artist = 'Love Fist')


In [ ]:
# 4. Write a Hive query that returns the total count and total price of all albums sold, grouped by
# genre and month.

select genre, month(transactiondate) month, sum(spendamount) netspend
from purchases
group by genre, month(transactiondate)

In [ ]:
# 5. Write Spark or PySpark code that returns the most recent album or single sold per artist.

df = sql("""select distinct aritst, album 
from albums a, (select songpurchaseid
from customerpurchses
where purchasedate = select (max(purchasedate) from customerpurchses group by artist)b
where a.songid = b.songpurchaseid
""")
df.show()

In [ ]:
#    6. Your database is in production and used by analysts. One of the analysts comes by your desk
#     and shows you this code. He wants to get the revenue and number of purchase transactions by
#     client for 2013 and 2014. How can you help to improve this query?
#     select 1 as in2013, AccountId, sum(USD) revenue, count(*) txns
#      from prod.PURCHASES
#      where DateTime between '2013-01-01' and '2013-12-31'
#      group by Accountid
#      union all
#     select 2 as in2014, AccountId, sum(USD) revenue, count(*) txns
#      from prod.PURCHASES
#      where DateTime between '2014-01-01' and '2013-12-31'
#      group by Accountid

select case when year(date_time) = 2013 then in2013 else in2014 end year, sum(USD) revenue, count(1) txns
from prod_PURCHASES
where DateTIme between  '2013-01-01' and '2014-12-31'
group by year(date_time)

In [ ]:
# 7. When building the ETL pipeline for this warehouse, what validations will you perform to ensure
# that the incoming data is clean and accurate? 

File Validations:
    1. Check the file is in utf-8 or converting them to utf is not causing issue
    2. Check for special characters in file
    3. Check if the received file confers to the agreed DIS with the client eg. column sequence, number of columns

Data Validation:
    1. Check whether the different keys confers to the business rules. eg. the customerid should be 10 digit number. So, check for if the id is not alphanumeric
    2. Check is the amount is not negative
    
    
Check for PII Data:
    1. Check whatever data is collected has been agreed with the client
    

In [ ]:
# 8. How will you ensure that the data can be queried with minimal time and resource consumption?

1. The Data modelling should be optimum. We should not unnecessarily normalize the tables as joining tables makes the query run slow
2. Use optimal Query - Take care of joins, filters etc
3. Create Queue in the cluster for each department so that they can use the cluster efficiently
4. Use dynamic spark submission to optimally use the cluster. 